In [2]:
# reveal.js presentation configuration
from notebook.services.config import ConfigManager

cm = ConfigManager()
cm.update('livereveal', {
              'theme': 'league',
              'transition': 'fade',
              'center': 'false',
              'overview' : 'true',
              'start_slideshow_at': 'selected'
})

# imports
import theano
from theano import tensor
import codecs
import numpy
import sys
from blocks import initialization
from blocks import roles
from blocks.model import Model
from blocks.bricks import Linear, NDimensionalSoftmax
from blocks.bricks.parallel import Fork
from blocks.bricks.recurrent import GatedRecurrent
from blocks.bricks.lookup import LookupTable
from blocks.filter import VariableFilter
from blocks.extensions.saveload import load_parameter_values
from blocks.bricks import NDimensionalSoftmax

# Language modeling with RNN

[Fabio A. González](http://dis.unal.edu.co/~fgonza/), Universidad Nacional de Colombia

## Setup
* Training data: Biblia Reina Valera 1960
* Software:
  * [Blocks](https://github.com/mila-udem/blocks): "Blocks is a framework that helps you build neural network models on top of Theano"
  * [Theano](http://deeplearning.net/software/theano/): "Theano is a Python library that allows you to define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently"

## Training data

In [3]:
# Load training file to get vocabulary
text_file = 'biblia.txt' # input file
with codecs.open(text_file, 'r', 'utf-8') as f:
    data = f.read()
    
chars = list(set(data))
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
print "Total number of chars:", len(data)
print "Vocabulary size:", vocab_size

Total number of chars: 978848
Vocabulary size: 85


## Training data

In [12]:
print data[21000:22000]

s será medido. 3 ¿Y por qué miras la paja que está en el ojo de tu hermano, y no echas de ver la viga que está en tu propio ojo? 4 ¿O cómo dirás a tu hermano: Déjame sacar la paja de tu ojo, y he aquí la viga en el ojo tuyo? 5 ¡Hipócrita! saca primero la viga de tu propio ojo, y entonces verás bien para sacar la paja del ojo de tu hermano.

6 No deis lo santo a los perros, ni echéis vuestras perlas delante de los cerdos, no sea que las pisoteen, y se vuelvan y os despedacen.

La oración, y la regla de oro

(Lc. 11.9-13; 6.31)

7 Pedid, y se os dará; buscad, y hallaréis; llamad, y se os abrirá. 8 Porque todo aquel que pide, recibe; y el que busca, halla; y al que llama, se le abrirá. 9 ¿Qué hombre hay de vosotros, que si su hijo le pide pan, le dará una piedra? 10 ¿O si le pide un pescado, le dará una serpiente? 11 Pues si vosotros, siendo malos, sabéis dar buenas dádivas a vuestros hijos, ¿cuánto más vuestro Padre que está en los cielos dará buenas cosas a los que le pidan? 12 


## Network architecture

<img src="rnn_architecture.jpg" width= 400>

## Define the layers

In [5]:
# Define the model structure
embedding_size = 256 # number of hidden units per layer

# Input
lookup = LookupTable(length=vocab_size, dim=embedding_size)

# Layer 1
fork1 = Fork(output_names=['linear1', 'gates1'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork1.name = 'fork1'
grnn1 = GatedRecurrent(dim=embedding_size)
grnn1.name = 'grnn1'

# Layer 2
fork2 = Fork(output_names=['linear2', 'gates2'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork2.name = 'fork2'
grnn2 = GatedRecurrent(dim=embedding_size)
grnn2.name = 'grnn2'

# Softmax layer
hidden_to_output = Linear(name='hidden_to_output', input_dim=embedding_size,
                          output_dim=vocab_size)
softmax = NDimensionalSoftmax()

## Connect the layers

In [6]:
# Propagate x until top brick to get y_hat predictions
x = tensor.imatrix('features')  # input
y = tensor.imatrix('targets')   # output
embedding = lookup.apply(x)
linear1, gates1 = fork1.apply(embedding)
h1 = grnn1.apply(linear1, gates1)
h1.name = 'h1'
linear2, gates2 = fork2.apply(h1)
h2 = grnn2.apply(linear2, gates2)
h2.name = 'h2'
linear3 = hidden_to_output.apply(h2)
linear3.name = 'linear3'
y_hat = softmax.apply(linear3, extra_ndim=1)
y_hat.name = 'y_hat'

# COST
cost = softmax.categorical_cross_entropy(y, linear3, extra_ndim=1).mean()
cost.name = 'cost'

model = Model(cost)

## Load parameters and build Theano graph

In [7]:
# Load model parameters from a file
model_file_name = 'model2.pkl'
model_params = model.get_parameter_dict().keys()
param_vals = {k:v for k,v in load_parameter_values(model_file_name).iteritems() if k in model_params}
model.set_parameter_values(param_vals)

In [8]:
# Define Theano graph
y, x = model.inputs
softmax = NDimensionalSoftmax()
linear_output = [v for v in model.variables if v.name == 'linear3'][0]
y_hat = softmax.apply(linear_output, extra_ndim=1)
predict = theano.function([x], y_hat)
theano.printing.pydotprint(predict, outfile="theano_graph.svg", format = 'svg', var_with_name_simple=True)  

The output file is available at theano_graph.svg


<img src="theano_graph.svg">

In [9]:
# Sampling function
def sample(nchars, text, temperature=1.0, seed=None):
    # represent the entry text as an array of indices 
    text = ''.join([ch for ch in text if ch in char_to_ix])
    x_curr = numpy.expand_dims(numpy.array([char_to_ix[ch] for ch in text], dtype='uint8'), axis=1)
    if seed:
        numpy.random.seed(seed)
    sample_string = '' + text
    for _ in range(nchars):
        probs = predict(x_curr).squeeze()[-1].astype('float64') # predicts the probability of the next char
        # based on the temperature decides whether to sample from the predicted probability distribution
        # or choose the char with the maximum probability
        if numpy.random.binomial(1, temperature) == 1: 
            probs = probs / probs.sum()
            sample = numpy.random.multinomial(1, probs).nonzero()[0][0]
        else:
            sample = probs.argmax()
        #sys.stdout.write('.')
        sample_string += ix_to_char[sample]
        if x_curr.shape[0] < 50:
            x_curr = numpy.roll(x_curr, -1)
            x_curr[-1] = sample
        else:
            x_curr = numpy.vstack((x_curr, [sample])).astype('int32')
    sys.stdout.write('\n')
    return sample_string

## Sampling from the model

* The model calculates the probability of the next word given the previous words:  
$$P(w_t | w_{t-1}, w_{t-2},\dots, w_{1})$$
* We sample from the model using this conditional probability
  ```python
  for i in [1..n]:
      P = predict_next() 
      bin_var = sample_binomial(temperature)
      if bin_var:
          w_i = sample_multinomial(P) 
      else:
          w_i = P.argmax() 
  ```

## Sampling from the model


In [18]:
print sample(500, u"Y Jesús dijo", temperature = 0.1)


Y Jesús dijo: ¿Qué harán los principales sacerdotes y de los que estaban en el camino a todos los que estaban en el camino a todos los que estaban el camino, y los que con él estaban lanzado en el templo de Dios, y de la tierra de la circuncisión es su carne, la cual es mi corazón estar con ellos, y los de los tributos para presentar a los que estaban en el camino a todos los que estaban en el camino a todos los misterios de la tierra de los cielos se apartan a los que estaban en el corazón de los hombres q


## Probability of a text
* The probability of a text is:  
$$P(w_1, \dots, w_n) = P(w_1)\prod_{i=2}^{n}\ P(w_i | w_{i-1},\dots, w_{1})$$


In [20]:
# Function to calculate the probability of a text
def log_likelihood(text):
    text = ''.join([ch for ch in text if ch in char_to_ix])
    x_curr = numpy.expand_dims(numpy.array([char_to_ix[ch] for ch in text], dtype='uint8'), axis=1)
    probs = predict(x_curr).squeeze()
    return sum([numpy.log(probs[i,c]) for i,c in enumerate(x_curr[1:].flatten())])

## Most likely phrases from a bag of words

In [21]:
from itertools import permutations
bow =  [u' ', 'hombre', 'ama', 'la', 'a']
perms = [' '.join(perm) for perm in permutations(bow)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[:20]:
    print p, t

27.0533799628 hombre hombre a la ama
27.0533799628 hombre hombre a la ama
28.0204834143 hombre hombre ama a la
28.0204834143 hombre hombre ama a la
28.9577404847 a hombre hombre la ama
28.9577404847 a hombre hombre la ama
29.6325383139 hombre a hombre la ama
29.6325383139 hombre a hombre la ama
30.1323473039 ama a hombre hombre la
30.1323473039 ama a hombre hombre la
30.8650613899 hombre hombre la ama a
30.8650613899 hombre hombre la ama a
31.1142256059 a hombre la hombre ama
31.1142256059 a hombre la hombre ama
31.2628376472 ama a hombre la hombre
31.2628376472 ama a hombre la hombre
31.4612043043 hombre ama a hombre la
31.4612043043 hombre ama a hombre la
31.7704667995 hombre hombre ama la a
31.7704667995 hombre hombre ama la a


## Least likely phrases 

In [12]:
perms = [' '.join(perm) for perm in permutations(bow)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[-20:]:
    print p, t

41.9514358657 mujer a la ama hombre
42.1607716451 mujer la hombre ama a
42.2761766982 la a hombre ama mujer
42.4480295589 hombre mujer la a ama
42.5642225358 ama hombre la a mujer
42.6927156776 mujer hombre la a ama
42.7488565983 la mujer a ama hombre
42.9256959793 ama hombre a mujer la
42.9949665482 ama la a hombre mujer
43.1326335912 mujer a ama la hombre
43.1956629937 mujer la a hombre ama
43.757676189 la ama hombre a mujer
44.0174657961 la a mujer ama hombre
44.6651412403 hombre la a ama mujer
46.1478429234 mujer ama la a hombre
46.1694953692 la a ama mujer hombre
46.5567532544 mujer la ama hombre a
47.7684307758 mujer a ama hombre la
53.1923222735 la a ama hombre mujer
57.9175873573 mujer la a ama hombre


## Morphology

In [13]:
from itertools import permutations
from random import shuffle


In [24]:
text = list(u'mnp')
perms = [''.join(perm) for perm in permutations(text)]
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[:5]:
    print p, t
print "------------------"
for p, t in sorted([(-log_likelihood(text),text) for text in perms])[-5:]:
    print p, t

11.6193473339 mpn
15.2685871124 nmp
20.0848698616 mnp
21.4446115494 npm
24.6974582672 pmn
------------------
15.2685871124 nmp
20.0848698616 mnp
21.4446115494 npm
24.6974582672 pmn
25.9515581131 pnm


## Structure

In [27]:
print sample(400, u"(Lc. ", temperature = 0.3)


(Lc. 12.23-39)

Muerte de los de Cataleza de los que estaba el campo para que estable decir, y no soy la palabra de los que estaba el señores de los que estaba en el fin de Simón habla entre vosotros falsos de los que estaba en el calor a los que éste hablas, como cardaba a tu carne, sino lo que estaba en el caminable demonio con todos los que estaban en la carne. 




















## A model trained from a different dataset

In [29]:
# Load training file to get vocabulary
text_file = 'reg1.txt' # input file
with codecs.open(text_file, 'r', 'utf-8') as f:
    data = f.read()
chars = [u'\u0432', u'\u1293', u'\u2014', u'\u2018', u'\u201c', u'\U00061ba1', u' ', u'$', u'(', u',', u'\xaf', u'\\', u'\u03b1', u'0', u'\xb3', u'4', u'\xb7', u'8', u'\xbb', u'\u203c', u'\xbf', u'@', u'\u9839', u'\xd7', u'\u263b', u'\u05e1', u'`', u'\xe3', u'd', u'\xe7', u'h', u'l', u'\xef', u'p', u'\xf3', u't', u'x', u'\ufffd', u'|', u'\ufeff', u'\u2003', u'j', u'\u200b', u'\U000ee28c', u'\u2013', u'\xa0', u'#', u"'", u'\xa8', u'+', u'/', u'\xb0', u'3', u'\xb4', u'7', u';', u'?', u'\U00065ca1', u'\u044b', u'\u2022', u'[', u'_', u'\u0161', u'\xe0', u'c', u'\U00061b65', u'\xe4', u'g', u'\xe8', u'k', u'\u2026', u'\xec', u'o', u's', u'\u0434', u'\xf4', u'w', u'{', u'\xfc', u'\U00061285', u'\u2030', u'\n', u'\ud299', u'\xa1', u'"', u'&', u'\xa9', u'*', u'\xad', u'.', u'\xb1', u'2', u'6', u':', u'\xbd', u'>', u'\u0436', u'\u2663', u'\u2665', u'^', u'\xe1', u'b', u'f', u'\xe9', u'\u266a', u'\xed', u'n', u'\xf1', u'r', u'v', u'\xf9', u'z', u'\xfd', u'\u266b', u'~', u'\t', u'\u2113', u'\x92', u'\u2019', u'\u201d', u'!', u'\u9ba0', u'\xa2', u'%', u'\u1ba4', u')', u'\xaa', u'-', u'\U0006192f', u'\xae', u'1', u'5', u'9', u'\xba', u'=', u'\u0441', u'\u0430', u'\U000ee825', u']', u'a', u'\xe2', u'e', u'i', u'\xea', u'm', u'q', u'\xf2', u'u', u'\u92a8', u'y', u'\xfa', u'}']
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
print "Total number of chars:", len(data)
print "Vocabulary size:", vocab_size

Total number of chars: 11120595
Vocabulary size: 152


In [30]:
# Input
lookup = LookupTable(length=vocab_size, dim=embedding_size)
# Layer 1
fork1 = Fork(output_names=['linear1', 'gates1'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork1.name = 'fork1'
grnn1 = GatedRecurrent(dim=embedding_size)
grnn1.name = 'grnn1'
# Layer 2
fork2 = Fork(output_names=['linear2', 'gates2'],
             input_dim=embedding_size, output_dims=[embedding_size, embedding_size * 2])
fork2.name = 'fork2'
grnn2 = GatedRecurrent(dim=embedding_size)
grnn2.name = 'grnn2'
# Softmax layer
hidden_to_output = Linear(name='hidden_to_output', input_dim=embedding_size,
                          output_dim=vocab_size)
softmax = NDimensionalSoftmax()

# Propagate x until top brick to get y_hat predictions
x = tensor.imatrix('features')  # input
y = tensor.imatrix('targets')   # output
embedding = lookup.apply(x)
linear1, gates1 = fork1.apply(embedding)
h1 = grnn1.apply(linear1, gates1)
h1.name = 'h1'
linear2, gates2 = fork2.apply(h1)
h2 = grnn2.apply(linear2, gates2)
h2.name = 'h2'
linear3 = hidden_to_output.apply(h2)
linear3.name = 'linear3'
y_hat = softmax.apply(linear3, extra_ndim=1)
y_hat.name = 'y_hat'

# COST
cost = softmax.categorical_cross_entropy(y, linear3, extra_ndim=1).mean()
cost.name = 'cost'

model = Model(cost)

# Load model parameters from a file
model_file_name = 'model4.pkl'
model_params = model.get_parameter_dict().keys()
param_vals = {k:v for k,v in load_parameter_values(model_file_name).iteritems() if k in model_params}
model.set_parameter_values(param_vals)

# Define Theano graph
y, x = model.inputs
softmax = NDimensionalSoftmax()
linear_output = [v for v in model.variables if v.name == 'linear3'][0]
y_hat = softmax.apply(linear_output, extra_ndim=1)
predict = theano.function([x], y_hat)

In [46]:
print sample(500, u"bienaventurado", temperature = 0.1, seed=10)


bienaventurado con la mano me la paso pensando en ti, no me hagas esperar
y a mi compañera

y siempre serás mejor que tu... receten y se mueren

y solo por un beso
y tu me miras y no te tengo
es mi bomba pa que se acabe la mano
y si tu quieres mami tu sabes que yo soy un tipo que no se acabe la mano
y si tu quieres mami tu sabes que yo soy tu juguete
y si tu te me estas tentando

y si tu quieres mami tu sabes que yo soy tu nene

y si tu te vas a encontrar la pared
tu y yo en ti me mata el dembow
es que tu me 


In [38]:
!grep -i "si tu quieres mami" reg.txt 


recuerda si tu quieres mami
recuerda si tu quieres mami
recuerda si tu quieres mami
